In [2]:
from multiprocessing import Process, Queue
import multiprocessing
from datasets.arrow_dataset import Dataset
from transformers import PreTrainedTokenizer
from typing import List, Dict
import torch
import random
import tqdm
import glob
from pathlib import Path
from datasets import concatenate_datasets, Dataset
from transformers import AutoTokenizer
#from model import load_pretrained_bert_base
from transformers import AutoModelForSequenceClassification, AutoModelForMaskedLM
#from model import get_bert_config, BertForSequenceClassificationWithLoss, BertForMaskedLMWithLoss
from transformers import BertConfig, BertForMaskedLM


/local/scratch-3/fwe21/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
import torch

torch.load("../../models/pretrained_bert.pt")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
dataset = load_dataset("glue","cola")

Dataset glue downloaded and prepared to /home/fwe21/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 439.90it/s]


In [14]:
dataset["test"]["label"]

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
tokenizer([["hey hello there", "what is going on"], ["peter is in the", "what is going on"]], max_length=20, padding="max_length", truncation=True)

{'input_ids': [[101, 4931, 7592, 2045, 102, 2054, 2003, 2183, 2006, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2848, 2003, 1999, 1996, 102, 2054, 2003, 2183, 2006, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [2]:
class BertOutput2(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, hidden_states: torch.Tensor, input_tensor: torch.Tensor) -> torch.Tensor:
        return torch.zeros((10,768))

In [3]:
torch.nn.Linear(5,5).weight

c:\Python310\lib\site-packages\torch\nn\modules\linear.py:96: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))


Parameter containing:
tensor([[-0.4093,  0.3513, -0.3729,  0.3773, -0.1935],
        [ 0.2439,  0.4271, -0.3410, -0.3140,  0.3044],
        [-0.3819, -0.2163,  0.2631,  0.2458, -0.1682],
        [-0.0460, -0.0471,  0.0888,  0.0803,  0.3447],
        [ 0.0013, -0.3685, -0.2319, -0.0751,  0.3551]], requires_grad=True)

In [9]:
import math
from torch import nn
from typing import Optional,Tuple
from transformers import BertConfig

class BertSelfAttention2(nn.Module):
    def __init__(self, config: BertConfig, position_embedding_type=None):
        super().__init__()
        assert position_embedding_type is None
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError("Hidden size not divisible by number of attention heads")
        
        self.num_heads = config.num_attention_heads
        self.head_size = config.hidden_size // self.num_heads

        self.query = nn.Linear(config.hidden_size, config.hidden_size)
        self.key = nn.Linear(config.hidden_size, config.hidden_size)
        self.value = nn.Linear(config.hidden_size, config.hidden_size)
        
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        assert head_mask is None
        assert encoder_hidden_states is None
        assert encoder_attention_mask is None
        assert past_key_value is None

        (batch_size, seq_len,hidden_size) = hidden_states.shape

        QKV_shape = (batch_size, seq_len, self.num_heads, self.head_size)
        Q = self.query(hidden_states).view(QKV_shape).permute((0,2,1,3))
        K = self.key(hidden_states).view(QKV_shape).permute((0,2,1,3))
        V = self.value(hidden_states).view(QKV_shape).permute((0,2,1,3))
        # Q,K,V are now shape [batch_size, self.num_heads, seq_len, head_size]

        attention_scores = Q @ K.permute((0,1,3,2)) # Shape is [batch_size, num_heads, seq_len, seq_len]
        if attention_mask is not None:
            attention_scores += attention_mask
        attention_probabilities = nn.functional.softmax(1/math.sqrt(self.head_size)*attention_scores, dim=-1)
        attention_probabilities = self.dropout(attention_probabilities)


        result = attention_probabilities @ V # Shape is [batch_size, num_heads, seq_len, head_size]
        result = result.permute((0,2,1,3)).reshape((batch_size, seq_len, hidden_size)) # Shape is [batch_size, seq_len, hidden_size]

        if output_attentions:
            return (result, attention_scores)
        else:
            return (result,)

In [10]:
config = BertConfig()
config.hidden_dropout_prob = 0
config.classifier_dropout = 0
config.attention_probs_dropout_prob = 0

In [14]:
from transformers.models.bert.modeling_bert import BertSelfAttention
from utils import set_random_seed

model1 = BertForMaskedLM(config)
state_dict = model1.state_dict()
model2 = BertForMaskedLM(config)
for layer in model2.bert.encoder.layer:
    layer.attention.self = BertSelfAttention2(config)
model2.load_state_dict(state_dict)

inp = torch.randint(200,1000, (10,30))
set_random_seed(0)
out1 = model1(inp)
set_random_seed(0)
out2 = model2(inp)

assert out1.logits == out2.logits

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [9]:


model = BertForMaskedLM(config)
state_dict = model.state_dict()
set_random_seed(0)
print(model())

MaskedLMOutput(loss=None, logits=tensor([[[ 0.0000, -0.0646,  0.2586,  ..., -0.4368, -0.6114,  0.1563],
         [ 0.0000, -0.8467, -0.4204,  ..., -0.8857, -0.2323,  0.5485],
         [ 0.0000, -0.3886,  0.2103,  ..., -1.0397, -0.4811,  0.8787]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [10]:
model.bert.encoder.layer[-1].attention.self = BertSelfAttention2(config)
model.load_state_dict(state_dict)
set_random_seed(0)
print(model(torch.tensor([[1,2,3]])))

MaskedLMOutput(loss=None, logits=tensor([[[ 0.0000, -0.0754,  0.2516,  ..., -0.4325, -0.6003,  0.1629],
         [ 0.0000, -0.8540, -0.4237,  ..., -0.8812, -0.2273,  0.5541],
         [ 0.0000, -0.3972,  0.2135,  ..., -1.0433, -0.4839,  0.8792]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [7]:
nn.functional.softmax(torch.rand((17,5,13,11)),dim=-1)[0][0][0].sum()

tensor(1.)

In [7]:
general_tinybert.config

BertConfig {
  "_name_or_path": "../../models/general_tinybert_huggingface/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [8]:
general_tinybert

AttributeError: 'BertForSequenceClassification' object has no attribute 'fit_dense'

In [4]:

general_small12h_sequence = torch.load("../../models/general_small12h.pt")
model = AutoModelForMaskedLM.from_config(get_bert_config("small12h"))
model.load_state_dict(general_small12h_sequence.state_dict(), strict=False)
torch.save(model,"../../models/general_small12h_mlm.pt")

In [4]:
AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny",num_labels=2).config

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [7]:
torch.save(bert_small,"../../models/pretrained_bert_small.pt")

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [13]:
len(tokenizer)

30522

: 

In [3]:
model = load_pretrained_bert_base()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
torch.save(model,"../../models/pretrained_bert.pt")

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.decode([2000,2001,2002])

'to was he'

In [ ]:

mlm_dataset_path = Path("../../wikipedia_mlm/")

datasets = [
    Dataset.load_from_disk(path)
    for path in sorted(glob.glob(str(mlm_dataset_path / "*")))
]

In [1]:
import torch

torch.argmax(torch.tensor([0,0,0]))

/local/scratch-3/fwe21/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor(2)

In [7]:
datasets

[Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 162948
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 172999
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 160956
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 175229
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 162693
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 168274
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 167903
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 161565
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 162859
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 165703
 }),
 Dataset({
     features: ['tokens', 'masked_token

In [9]:
import os
"\n".join(os.popen('free -t -m').readlines())

['              total        used        free      shared  buff/cache   available\n',
 'Mem:         128800       41259       33827           5       53713       86274\n',
 'Swap:          1766        1520         245\n',
 'Total:       130566       42780       34072\n']

In [35]:
import os, psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss//1000000)

1689


In [13]:
dataset = concatenate_datasets(datasets)

Transform <function _concatenate_map_style_datasets at 0x7f33d57a59d0> couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [21]:
dataset.set_format("torch")

In [31]:
x = 0
for i in tqdm.tqdm(range(1000)):
    x += len(dataset[random.randint(0,len(dataset)-1)])

100%|██████████| 1000/1000 [00:11<00:00, 84.48it/s]


3000

In [8]:
import random
def f(n):
    l = [0]*n

    for i in range(n):
        l[random.randint(0,n-1)]+=1
    return max(l)

In [9]:
max(l)

7